In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"


In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
# web scraping
# imrt urllib.request
from bs4 import BeautifulSoup
import re
#Data and analysis
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
from sklearn import preprocessing, linear_model, metrics
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_row', 1000)

%matplotlib inline
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 6),
         'axes.labelsize': 'x-large',
         'axes.titlesize': 20,
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [5]:
df = pd.read_csv('/Users/natalie/Desktop/GA/project_3_data/df-cleaned.csv')

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

In [6]:
df.head()

,Job,Company,Location,Description,Starting Salary ($),Ending Salary ($),State,Average Salary ($)
0,DATA SCIENTIST,First Derivatives,New York,Data Scientists explore vast amounts of inform...,70000.0,80000.0,NY,75000.0
1,DATA SCIENTIST,MetroPlus Health Plan,New York,The Data Scientist will be tasked with leading...,90000.0,115000.0,NY,102500.0
2,DATA SCIENTIST,Enterprise Select,New York,"Financial Services data such as transactions, ...",185000.0,185000.0,NY,185000.0
3,DATA ANALYST,POLICE DEPARTMENT,New York,"Extensive knowledge of applied statistics, ana...",70286.0,88213.0,NY,79249.5
4,DATA SCIENTIST - NEURAL NETWORKS,Geode Executive Search,New York,A well-known tech company in New York is looki...,130000.0,130000.0,NY,130000.0


In [7]:
df.shape

(424, 8)

In [8]:
df.describe()

,Starting Salary ($),Ending Salary ($),Average Salary ($)
count,424.000000,424.000000,424.000000
mean,111303.523585,139796.660377,125550.091981
std,43629.959843,54853.719907,47555.108835
min,20000.000000,25000.000000,22500.000000
25%,76320.500000,100000.000000,90000.000000
50%,110000.000000,140000.000000,125000.000000
75%,140000.000000,180000.000000,160000.000000
max,300000.000000,309750.000000,300000.000000


In [9]:
median = df['Average Salary ($)'].median()

In [10]:
min = df['Average Salary ($)'].min()

In [11]:
median

125000.0

In [12]:
def greater_med(m):
    if m >= median:
        return 1
    return 0

df['Above Median 125k'] = df['Average Salary ($)'].apply(greater_med)

In [13]:
df.head()

,Job,Company,Location,Description,Starting Salary ($),Ending Salary ($),State,Average Salary ($),Above Median 125k
0,DATA SCIENTIST,First Derivatives,New York,Data Scientists explore vast amounts of inform...,70000.0,80000.0,NY,75000.0,0
1,DATA SCIENTIST,MetroPlus Health Plan,New York,The Data Scientist will be tasked with leading...,90000.0,115000.0,NY,102500.0,0
2,DATA SCIENTIST,Enterprise Select,New York,"Financial Services data such as transactions, ...",185000.0,185000.0,NY,185000.0,1
3,DATA ANALYST,POLICE DEPARTMENT,New York,"Extensive knowledge of applied statistics, ana...",70286.0,88213.0,NY,79249.5,0
4,DATA SCIENTIST - NEURAL NETWORKS,Geode Executive Search,New York,A well-known tech company in New York is looki...,130000.0,130000.0,NY,130000.0,1


In [14]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [15]:
# ## YOUR CODE HERE
# It is 50/50 becuase we are spliting on the median 

In [16]:
df['Location'].nunique()

94

# getting dummies for local & Intial random trees

In [17]:
def get_dummy_locs(data):
    return pd.get_dummies(data)

State_df = pd.concat([get_dummy_locs(df['State'])], axis=1)
location_df = pd.concat([get_dummy_locs(df['Location']),get_dummy_locs(df['State'])], axis=1)


In [62]:
def eda(dataframe):
    # This is the EDA function Ritika wrote and showed us. It is wonderful.
    print "Missing Values \n \n", dataframe.isnull().sum(),"\n"
    print "Duplicate Rows \n", dataframe.duplicated().sum(),"\n" # Added this
    print "Dataframe Types \n \n", dataframe.dtypes,"\n"
    print "Dataframe Shape \n", dataframe.shape,"\n"
    print "Dataframe Describe \n \n", dataframe.describe(include='all'),"\n"
    for feature in dataframe: # Prints unique values for each column
        print feature
        print dataframe[feature].nunique()

In [63]:
eda(State_df)

Missing Values 
 
AL    0
AZ    0
CA    0
CO    0
DC    0
DE    0
FL    0
GA    0
IL    0
MA    0
MD    0
MI    0
MN    0
MO    0
NC    0
NV    0
NY    0
OH    0
OK    0
OR    0
PA    0
TN    0
TX    0
UT    0
VA    0
WA    0
dtype: int64 

Duplicate Rows 
398 

Dataframe Types 
 
AL    uint8
AZ    uint8
CA    uint8
CO    uint8
DC    uint8
DE    uint8
FL    uint8
GA    uint8
IL    uint8
MA    uint8
MD    uint8
MI    uint8
MN    uint8
MO    uint8
NC    uint8
NV    uint8
NY    uint8
OH    uint8
OK    uint8
OR    uint8
PA    uint8
TN    uint8
TX    uint8
UT    uint8
VA    uint8
WA    uint8
dtype: object 

Dataframe Shape 
(424, 26) 

Dataframe Describe 
 
               AL          AZ          CA          CO          DC          DE  \
count  424.000000  424.000000  424.000000  424.000000  424.000000  424.000000   
mean     0.002358    0.016509    0.238208    0.007075    0.049528    0.004717   
std      0.048564    0.127575    0.426490    0.083917    0.217225    0.068599   
min      0.0000

In [18]:
## YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from bs4 import BeautifulSoup
import pandas as pd
# import requests
import xgboost
import re


/Users/natalie/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
location_df.head()

,Alexandria,Arlington,Atlanta,Auburn,Austin,Baltimore,Bethesda,Beverly Hills,Birmingham,Boston,Boulder,Brisbane,Brooklyn,Calabasas,Campbell,Cary,Charlotte,Chicago,Cincinnati,Clayton,Cleveland,Columbia,Coral Gables,Dallas,Dania Beach,...,AZ,CA,CO,DC,DE,FL,GA,IL,MA,MD,MI,MN,MO,NC,NV,NY,OH,OK,OR,PA,TN,TX,UT,VA,WA
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## First lets random forest with just state dummies, this will be useful for viz later.

In [20]:
y = df['Above Median 125k']
X = State_df
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)

In [21]:
dt = RandomForestClassifier(class_weight='balanced')
dt.fit(X,y)
s = cross_val_score(dt, X, y, cv=cv)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest with Balanced Classes", s.mean().round(3), s.std().round(3))

Random Forest with Balanced Classes Score:	0.651 ± 0.04


In [22]:
importance_dataframe_big = pd.DataFrame(dt.feature_importances_, X.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe_big.to_csv('States_for_Tabby.csv',index = False)
importance_dataframe_big.head(100)

,importance
CA,0.213731
GA,0.145161
MD,0.120197
FL,0.079320
IL,0.059136
TX,0.052173
NY,0.039189
UT,0.030015
NC,0.024136
AZ,0.023882


## now lets try it with both location dummies and see if it improves our score and feature importance

In [23]:
y = df['Above Median 125k']
X = location_df
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)

In [24]:
dt = RandomForestClassifier(class_weight='balanced')
dt.fit(X,y)
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest with Balanced Classes", s.mean().round(3), s.std().round(3))

Random Forest with Balanced Classes Score:	0.738 ± 0.04


In [25]:
importance_dataframe_big1 = pd.DataFrame(dt.feature_importances_, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe_big1.head(100)

,importance
New York,0.089585
CA,0.077985
San Francisco,0.075359
San Jose,0.038242
GA,0.034815
Charlotte,0.026100
NY,0.025505
Atlanta,0.025169
Queens,0.024524
Boston,0.024499


# Lets model with some job title keyword dummies
## using frequency I went with lead, senior, machine,python

In [26]:
df_boo_jobs = df
df_boo_jobs['Lead'] = df_boo_jobs['Job'].str.contains('LEAD')==True

In [27]:
df_boo_jobs

,Job,Company,Location,Description,Starting Salary ($),Ending Salary ($),State,Average Salary ($),Above Median 125k,Lead
0,DATA SCIENTIST,First Derivatives,New York,Data Scientists explore vast amounts of inform...,70000.0,80000.0,NY,75000.0,0,False
1,DATA SCIENTIST,MetroPlus Health Plan,New York,The Data Scientist will be tasked with leading...,90000.0,115000.0,NY,102500.0,0,False
2,DATA SCIENTIST,Enterprise Select,New York,"Financial Services data such as transactions, ...",185000.0,185000.0,NY,185000.0,1,False
3,DATA ANALYST,POLICE DEPARTMENT,New York,"Extensive knowledge of applied statistics, ana...",70286.0,88213.0,NY,79249.5,0,False
4,DATA SCIENTIST - NEURAL NETWORKS,Geode Executive Search,New York,A well-known tech company in New York is looki...,130000.0,130000.0,NY,130000.0,1,False
5,GEOSPATIAL DATA SCIENTIST,DEPARTMENT OF FINANCE,Manhattan,The Property Valuation &amp; Mapping Unit is s...,70286.0,81000.0,NY,75643.0,0,False
6,CLIENT FACING PYTHON ENGINEERS FOR A.I DATA CO...,Birch & James Associates Limited,New York,An interest in machine learning and/or data sc...,70000.0,110000.0,NY,90000.0,0,False
7,DATA MANAGER/STATISTICIAN,Stony Brook University,Stony Brook,Compliant data analysis. Biostatistics and dat...,52000.0,62000.0,NY,57000.0,0,False
8,DATA ANALYST/MODELER,DEPARTMENT OF FINANCE,Brooklyn,"Strong programming, data analysis, statistical...",70286.0,80829.0,NY,75557.5,0,False
9,POLICY DATA ANALYST,DEPARTMENT OF FINANCE,Manhattan,"Strong quantitative and research skills, inclu...",70286.0,80829.0,NY,75557.5,0,False


In [28]:
df_boo_jobs = df
df_boo_jobs['Senior'] = df_boo_jobs['Job'].str.contains('SENIOR')==True

In [29]:
df_boo_jobs

,Job,Company,Location,Description,Starting Salary ($),Ending Salary ($),State,Average Salary ($),Above Median 125k,Lead,Senior
0,DATA SCIENTIST,First Derivatives,New York,Data Scientists explore vast amounts of inform...,70000.0,80000.0,NY,75000.0,0,False,False
1,DATA SCIENTIST,MetroPlus Health Plan,New York,The Data Scientist will be tasked with leading...,90000.0,115000.0,NY,102500.0,0,False,False
2,DATA SCIENTIST,Enterprise Select,New York,"Financial Services data such as transactions, ...",185000.0,185000.0,NY,185000.0,1,False,False
3,DATA ANALYST,POLICE DEPARTMENT,New York,"Extensive knowledge of applied statistics, ana...",70286.0,88213.0,NY,79249.5,0,False,False
4,DATA SCIENTIST - NEURAL NETWORKS,Geode Executive Search,New York,A well-known tech company in New York is looki...,130000.0,130000.0,NY,130000.0,1,False,False
5,GEOSPATIAL DATA SCIENTIST,DEPARTMENT OF FINANCE,Manhattan,The Property Valuation &amp; Mapping Unit is s...,70286.0,81000.0,NY,75643.0,0,False,False
6,CLIENT FACING PYTHON ENGINEERS FOR A.I DATA CO...,Birch & James Associates Limited,New York,An interest in machine learning and/or data sc...,70000.0,110000.0,NY,90000.0,0,False,False
7,DATA MANAGER/STATISTICIAN,Stony Brook University,Stony Brook,Compliant data analysis. Biostatistics and dat...,52000.0,62000.0,NY,57000.0,0,False,False
8,DATA ANALYST/MODELER,DEPARTMENT OF FINANCE,Brooklyn,"Strong programming, data analysis, statistical...",70286.0,80829.0,NY,75557.5,0,False,False
9,POLICY DATA ANALYST,DEPARTMENT OF FINANCE,Manhattan,"Strong quantitative and research skills, inclu...",70286.0,80829.0,NY,75557.5,0,False,False


In [30]:
df_boo_jobs = df
df_boo_jobs['Machine'] = df_boo_jobs['Job'].str.contains('MACHINE')==True

In [31]:
df_boo_jobs

,Job,Company,Location,Description,Starting Salary ($),Ending Salary ($),State,Average Salary ($),Above Median 125k,Lead,Senior,Machine
0,DATA SCIENTIST,First Derivatives,New York,Data Scientists explore vast amounts of inform...,70000.0,80000.0,NY,75000.0,0,False,False,False
1,DATA SCIENTIST,MetroPlus Health Plan,New York,The Data Scientist will be tasked with leading...,90000.0,115000.0,NY,102500.0,0,False,False,False
2,DATA SCIENTIST,Enterprise Select,New York,"Financial Services data such as transactions, ...",185000.0,185000.0,NY,185000.0,1,False,False,False
3,DATA ANALYST,POLICE DEPARTMENT,New York,"Extensive knowledge of applied statistics, ana...",70286.0,88213.0,NY,79249.5,0,False,False,False
4,DATA SCIENTIST - NEURAL NETWORKS,Geode Executive Search,New York,A well-known tech company in New York is looki...,130000.0,130000.0,NY,130000.0,1,False,False,False
5,GEOSPATIAL DATA SCIENTIST,DEPARTMENT OF FINANCE,Manhattan,The Property Valuation &amp; Mapping Unit is s...,70286.0,81000.0,NY,75643.0,0,False,False,False
6,CLIENT FACING PYTHON ENGINEERS FOR A.I DATA CO...,Birch & James Associates Limited,New York,An interest in machine learning and/or data sc...,70000.0,110000.0,NY,90000.0,0,False,False,False
7,DATA MANAGER/STATISTICIAN,Stony Brook University,Stony Brook,Compliant data analysis. Biostatistics and dat...,52000.0,62000.0,NY,57000.0,0,False,False,False
8,DATA ANALYST/MODELER,DEPARTMENT OF FINANCE,Brooklyn,"Strong programming, data analysis, statistical...",70286.0,80829.0,NY,75557.5,0,False,False,False
9,POLICY DATA ANALYST,DEPARTMENT OF FINANCE,Manhattan,"Strong quantitative and research skills, inclu...",70286.0,80829.0,NY,75557.5,0,False,False,False


In [32]:
df_boo_jobs = df
df_boo_jobs['Python'] = df_boo_jobs['Job'].str.contains('PYTHON')==True

In [33]:
df_boo_jobs

,Job,Company,Location,Description,Starting Salary ($),Ending Salary ($),State,Average Salary ($),Above Median 125k,Lead,Senior,Machine,Python
0,DATA SCIENTIST,First Derivatives,New York,Data Scientists explore vast amounts of inform...,70000.0,80000.0,NY,75000.0,0,False,False,False,False
1,DATA SCIENTIST,MetroPlus Health Plan,New York,The Data Scientist will be tasked with leading...,90000.0,115000.0,NY,102500.0,0,False,False,False,False
2,DATA SCIENTIST,Enterprise Select,New York,"Financial Services data such as transactions, ...",185000.0,185000.0,NY,185000.0,1,False,False,False,False
3,DATA ANALYST,POLICE DEPARTMENT,New York,"Extensive knowledge of applied statistics, ana...",70286.0,88213.0,NY,79249.5,0,False,False,False,False
4,DATA SCIENTIST - NEURAL NETWORKS,Geode Executive Search,New York,A well-known tech company in New York is looki...,130000.0,130000.0,NY,130000.0,1,False,False,False,False
5,GEOSPATIAL DATA SCIENTIST,DEPARTMENT OF FINANCE,Manhattan,The Property Valuation &amp; Mapping Unit is s...,70286.0,81000.0,NY,75643.0,0,False,False,False,False
6,CLIENT FACING PYTHON ENGINEERS FOR A.I DATA CO...,Birch & James Associates Limited,New York,An interest in machine learning and/or data sc...,70000.0,110000.0,NY,90000.0,0,False,False,False,True
7,DATA MANAGER/STATISTICIAN,Stony Brook University,Stony Brook,Compliant data analysis. Biostatistics and dat...,52000.0,62000.0,NY,57000.0,0,False,False,False,False
8,DATA ANALYST/MODELER,DEPARTMENT OF FINANCE,Brooklyn,"Strong programming, data analysis, statistical...",70286.0,80829.0,NY,75557.5,0,False,False,False,False
9,POLICY DATA ANALYST,DEPARTMENT OF FINANCE,Manhattan,"Strong quantitative and research skills, inclu...",70286.0,80829.0,NY,75557.5,0,False,False,False,False


# Lets get our model going with our new jobtitle model

In [34]:
location_job_df = pd.concat([location_df,df_boo_jobs['Machine'],df_boo_jobs['Python'],df_boo_jobs['Senior'],df_boo_jobs['Lead']], axis=1)

In [35]:
y = df['Above Median 125k']
X = location_job_df
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)


In [36]:
dt = RandomForestClassifier(class_weight='balanced')
dt.fit(X,y)
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest with Balanced Classes", s.mean().round(3), s.std().round(3))

Random Forest with Balanced Classes Score:	0.715 ± 0.01


In [37]:
importance_dataframe_big2 = pd.DataFrame(dt.feature_importances_, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe_big2.head(20)

,importance
Senior,0.080304
Machine,0.079027
San Francisco,0.079017
New York,0.068124
CA,0.063292
Lead,0.046756
Charlotte,0.034999
NY,0.028116
Python,0.027291
Boston,0.017829


## on average of our forests we imporved our model a little bit by including our job keywords

# Vectorizing text

### vec_ing Description 

In [38]:
vec_desc = CountVectorizer(stop_words='english', min_df=12)
vec_desc.fit(df.Description)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=12,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [39]:
vec_desc_df = vec_desc.transform(df.Description)
# trnasoform here deals with each column individually


In [40]:
df_desc_words = pd.DataFrame(vec_desc_df.todense(), columns=vec_desc.get_feature_names())
df_desc_words
print df_desc_words.shape

(424, 81)


In [41]:
# word_counts = df_desc_words.sum(axis=0)
# word_counts.sort_values(ascending = False).head(82)
# # 139 is 10% rounded up to prevent cutting off at the same value exported to csv

### vec_ing Job title


In [42]:
vec_job = CountVectorizer(stop_words='english',min_df=10)
vec_job.fit(df.Job)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [43]:
vec_job_df = vec_job.transform(df.Job)
# trnasoform here deals with each column individually


In [44]:
df_job_words = pd.DataFrame(vec_job_df.todense(), columns=vec_job.get_feature_names())
df_job_words
print df_job_words.shape

(424, 22)


In [45]:
#  word_counts = df_job_words.sum(axis=0)
#  word_counts.sort_values(ascending = False).head(27)
# # # 27 is 10% rounded up to prevent cutting off at the same value exported to csv

## vec_ing company (becuase why not?)

In [46]:
vec_comp = CountVectorizer(stop_words='english',min_df=10)
vec_comp.fit(df.Company)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [47]:
vec_comp_df = vec_comp.transform(df.Company)
# trnasoform here deals with each column individually


In [48]:
df_comp_words = pd.DataFrame(vec_comp_df.todense(), columns=vec_comp.get_feature_names())
df_comp_words
print df_comp_words.shape

(424, 25)


In [49]:
#  word_counts = df_comp_words.sum(axis=0)
#  word_counts.sort_values(ascending = False).head(25)
# # # # 26 is 10% rounded up to prevent cutting off at the same value exported to csv

### Concat all the vecs and the local dummies

In [50]:
vec_words = pd.concat([location_df,df_desc_words, df_job_words, df_comp_words], axis=1) #
model_predictor = vec_words
model_predictor = model_predictor.loc[:, (model_predictor != 0).any(axis=0)] ## Drop any columns that contain only 0
model_predictor.shape


(424, 248)

# now lets model with random tress using all of our vecs and local dummies

In [51]:
y = df['Above Median 125k']
X = model_predictor
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)


In [52]:
dt = RandomForestClassifier(class_weight='balanced')
dt.fit(X,y)
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest with Balanced Classes", s.mean().round(3), s.std().round(3))

Random Forest with Balanced Classes Score:	0.766 ± 0.061


In [53]:
importance_dataframe_big3 = pd.DataFrame(dt.feature_importances_, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe_big3['Feat'] = importance_dataframe_big3.index
importance_dataframe_big3.to_csv('All feat tabby.csv',index = False)
importance_dataframe_big3.head(20)



,importance,Feat
analyst,0.096344,analyst
CA,0.038309,CA
data,0.028970,data
research,0.027710,research
senior,0.026646,senior
learning,0.019145,learning
engineer,0.017566,engineer
New York,0.016169,New York
scientist,0.015358,scientist
scientist,0.014770,scientist


#### Repeat the model-building process with a non-tree-based method.

In [54]:
from sklearn.svm import SVC
model_svmrbf = SVC(kernel='rbf')
scores_svm = cross_val_score(model_svmrbf, X, y, cv=6)
print 'Cross-validated scores:', scores_svm
print 'Average score:', scores_svm.mean()
print 'Standard deviation of score:', scores_svm.std()


Cross-validated scores: [ 0.73611111  0.66197183  0.6056338   0.8         0.98571429  0.74285714]
Average score: 0.755381362248
Standard deviation of score: 0.120223367947


In [55]:
model_svmlm = SVC(kernel='linear')
scores_svmlm = cross_val_score(model_svmlm, X, y, cv=6)
print 'Cross-validated scores:', scores_svmlm
print 'Average score:', scores_svmlm.mean()
print 'Standard deviation of score:', scores_svmlm.std()


Cross-validated scores: [ 0.70833333  0.70422535  0.76056338  0.65714286  0.8         0.64285714]
Average score: 0.712187010955
Standard deviation of score: 0.0547206884648


In [56]:
## YOUR CODE HERE